In [ ]:
pip install --upgrade pip
pip install lightgbm


In [46]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib
import lightgbm as lgb

ModuleNotFoundError: No module named 'lightgbm'

In [7]:
train_dir = r'C:\Users\FreeComp\OneDrive\Desktop\Alzheimer_s Dataset\Alzheimer_s Dataset\train'  
test_dir = r'C:\Users\FreeComp\OneDrive\Desktop\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [9]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [11]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 5121 images belonging to 4 classes.


In [30]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1279 images belonging to 4 classes.


In [32]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

In [34]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

161/161 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step


In [36]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step


In [44]:
# Use LightGBM 
# استيراد المكتبة


# إعداد نموذج LightGBM
lgb_classifier = lgb.LGBMClassifier(
    boosting_type='gbdt',  # Use gradient boosting
    num_leaves=31,         # Maximum leaves in one tree
    max_depth=-1,          # No maximum depth
    learning_rate=0.1,     # Step size
    n_estimators=100,      # Number of trees
    objective='multiclass',    # multi classification
)

# تدريب النموذج
lgb_classifier.fit(train_features, train_labels)




ModuleNotFoundError: No module named 'lightgbm'

In [40]:
# Make predictions on the test data
test_predictions = lgb_classifier.predict(test_features)

NameError: name 'lgb_classifier' is not defined

In [28]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

NameError: name 'accuracy_score' is not defined

In [28]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor8.h5')

In [29]:
# Save the trained LightGBM model
joblib.dump(lgb_classifier, 'lgb_classifier.pkl')

['lgb_classifier.pkl']

In [30]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor8.h5')
dt_classifier = joblib.load('lgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for lightgbm

    # Classify the features with the lightgbm model
    prediction = lgb_classifier.predict(features)

    # Ensure prediction is an integer
    predicted_class = int(prediction[0])
    
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (62).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
The image is classified as: NonDemented
